<h1 style="color:black" align="center">Представления для текстов</h1>

## Bag of words

* Заводим словарь, состоящий из всех слов в выборке
* Делаем признак-индикатор для каждого слова из словаря
* Можно добавлять n-граммы

https://towardsdatascience.com/from-word-embeddings-to-pretrained-language-models-a-new-age-in-nlp-part-1-7ed0c7f3dfc5

<img src='img/lecture09/1.png'>

### Недостатки:

* Слишком много признаков
* Не учитываем смыслы слов
* Семантически похожие тексты могут иметь очень разные представления

<h1 style="color:#008B8B">1. word2vec</h1>

* Попробуем обучить вектор-представление для каждого слова
* Что потребовать от такого представления?


* Важная идея: если выкинуть слово, то оно должно хорошо восстанавливаться по представлениям соседних слов
* Может применять и при работе с изображениями

<img src='img/lecture09/2.png'>

https://arxiv.org/abs/1301.3781

# 1.1 Skip-gram model

Вероятность встреить слово $w_O$ рядом со словом $w_I$:

$$\large P(w_O | w_I) = 
\frac{\exp (\langle v_{w_O}^{'}, v_{w_I} \rangle)}
{\sum\limits_{w \in W} \exp(\langle v_{w}^{'}, v_{w_I} \rangle)}$$

* $W$ - словарь

Для каждого слова будем выучивать два вектора:

* $v_w$ - "Центральное" представление слова. В случае, когда слово стоит в центре контекста и по нему предсказыаем соседние слова.
* $v_{w}^{'}$ - "Контекстное" представление слова. Слово стоит вне контекста и его предсказываем по центральному слову.

**Зачем выучивать два вектора?** Мы для каждого слова выучиваем два вектора, если слово является центральным и вектор, если слово стоит в контексте. Так работает лушче, можно было бы выучивать всего один вектор и использовать все без штрихов.

### Разбор формулы:

Cлово $w_O$ находится в контексте слова $w_I$, если между ними расстояние не больше некоторого чилса. Контекстом центарльного слова $w_I$ называется соседние $k$ слов (два слова до, два после).

$P(w_O | w_I)$ - вероятность того, что встретим слово $w_O$ в контексте слова $w_I$. Ну эта вероятность считается по формле выше:

1. В числителе берём скалярное произведение центального представления слова $w_I$ (для слова $w_I$ берём вектор, который предсказывает что рядом). Для слова $w_O$ берём контекстное представление. Получаем характеристику соноправленности двух векторов. Берём от этого экспоненту, так как скалярное произведение может быть отрицательным и положительными.

2. И делим на сумму экспонент, где берём выходные векторы для всех слов. Это для нормировки, чтобы сумма всех вероятностей была равна 1, это просто SoftMax. Чем более соноправлены векторы двух слов, тем выще верояность того, что они встретятся вместе.

При максимизации экспонента от скалярного произведения должна быть как можно больше, значит мы требуем чтобы само скалярное произведение было как можно больше. И при максимизации скалаярного произведения мы требуем соноправленности векторов. Если два слова встречаются рядом, то их векторы будут как можно более соноправлены.

### Функционал

Функционал для текста $T = (w_1, w_2, \ldots, w_n)$ из $n$-слов:

$$\large \sum\limits_{i=1}^{n} \sum\limits_{-c \le j \le c \\ \quad j\ne 0 \\ 1 \le i + c \le n} \log P(w_{i + j} | w_j) \to \max$$

1. Суммируем по всем словам в тексте, берём $i$-е слово.
2. Суммируем по контексту, где $c$ - это размер контекста.
3. Под суммой стоит вероятность встретить слово $w_{i +j}$ в контексте слова $w_i$, берём логарифм этой вероятности. 

Простоми словами, берём $i$ слово и требуем, чтобы вероятность встреить другие слова в его контексте была максимальна, только для слов из контекста.

Максимизуерм по векторам слов $\large v_{w_1}^{'}, \ldots, v_{w_n}^{'}$ и по $\large v_{w_1}, \ldots, v_{w_n}$. Подбираем векторы слов так, чтобы вероятность встретить слово в его контексте была как можно больше.

**Почему логарифм?** Мы записываем правдоподобие выборки, для этого небоходимо перемножать вероятности. А для того, чтобы мы их суммировали необходимо вязть логарифм.

### Проблема формулы:

$$\large P(w_O | w_I) = 
\frac{\exp (\langle v_{w_O}^{'}, v_{w_I} \rangle)}
{\sum\limits_{w \in W} \exp(\langle v_{w}^{'}, v_{w_I} \rangle)}$$

* Считать знаменатель ОЧЕНЬ затратно
* Значит, и производные считать тоже долго

Необходимо избавиться от суммы, хочется чтобы вероятность не зависила от всего словаря.

# 1.2 Negative sampling

Заменим вероятность следующей формулой:

$$\large P(w_O | w_I) = \log \sigma (\langle v_{w_O}^{'}, v_{w_I} \rangle) + \sum\limits_{i=1}^k \log \sigma (-\langle v_{w_i}^{'}, v_{w_I} \rangle) \to \max$$

* $w_i$ — случайно выбранные слова
* Слово $w$ генерируется с вероятностью $P(w)$ — шумовое распределение
* $\large P(w) = \frac{U(w)^{\frac{3}{4}}}{\sum\limits_{v \in W} U(v)^{\frac{3}{4}}}$, $U(v)$ — частота слова $v$ в корпусе текстов


1. Мы взяли два слова $w_I, w_O$. Берём сигмойду от скалярногоо произведения векторов двух слов и оно должно быть как можно больше. И это требование уже требует чтобы скалярное произвдение было как можно больше.

В прошлой формуле, мы требовали, чтобы экспонента от скалярного произведения должна быть как можно больше. Если увеличивали скалярное произвдение для слова $w_O$ в контексте $w_I$, тогда уменьшались скалярные произвдения для $w_I$ со всеми другими словами, так как все вероятности суммируются в еденицу. Следовательно, когда мы настраиваем вероятность для одного слова, настраивались вероятности на всех других словах. 

И получается, только для тех слов, которые встречались рядом, вероятности большие. В Negative sampling мы потеряли данное свойство, мы просто требуем чтобы сигмойда от скалярного произведния была как можно больше. Добавим ещё одно слогаемое в функционал на который мы обучаем предсталвения:

2. Сгенерируем для $w_I$ случайные слова из словаря и потребуем, чтобы для $w_i$ сигмойда от скалярного произведения была как можно меньше. Так как мы максимизируем, добавим минус.

Тем самым мы избавились от нормировки, при этом будем минимизировать вероятности для из не контекста.

### Особенности обучения:

* Положительные примеры — слова, стоящие рядом
*  Отрицательные примеры: подбираем к слову «шум», то есть другое слово, которое не находится рядом
*  Важно семплировать в SGD слова с учётом их популярности — иначе будем обучаться только на самые частые слова

### Как это использовать?

* Можно искать похожие слова
* Можно менять формы слов
* Можно искать определённые отношения
* Можно использовать как признаки для моделей

# word2vec

* Яркий пример self-supervision
* Сейчас находит применения для изображения и даже для табличных данных
* Оказывается, в данных очень много информации даже без разметки

### Проблемы word2vec

* Не учитываем структуру слов
* Не закладываем никакой априорной информации о разных формах одного слова
* Не умеем обрабатывать опечатки


<h1 style="color:#008B8B">2. FastText</h1>

& Заменим каждое слово на «мешок»
* «руслан» $\to$ (<руслан>, <ру, рус, усл, сла, лан, ан>)
* Слово $\large w$ заменяется на набор токенов $\large t_1, \ldots, t_n$
* Мы обучаем векторы токенов: $\large v_{t_1}, \ldots, v_{t_n}$ (на самом деле есть «центральные» и «контекстные» версии всех векторов)
* $\large z_w = \sum\limits_{i=1}^n v_{t_i}$ — вектор слова складывается из подслов.
* Все остальные детали — как в word2vec

<h1 style="color:black" align="center">Работа с текстом</h1>

Научились строить векторы отдельных слов. Как это использовать для решения задач на текстах?

* Векторные представления строятся для слов
* Можно просто усреднить по всем словам — получим признаки для текста
* Можно усреднять с весами
* Можно ли умнее?

<h1 style="color:#008B8B">1. CNN для последовательностей</h1>

Имеется некоторое предложение. Для каждого слова возьмём его вектор. Получаем матрицу число слов на размер векторых представлений.
 
<img src='img/lecture09/3.png'>

Дальше мы применяем свёртки. Берём фильр, размера два на число компонент вектора слов. И проходимся по всей матрице, получаем один канал. Полсе можно взять ещё один фильтр другого размера (по двум словама, трём...), получим ещё одни канал и так далее...

При этом, свёртка должна быть полной по размеру векторного представления слов.

Псоле берём каждый канал и записываем максимум, так как из-за того, что мы можем применять свёртки разных рамзеров, выходные каналы будут получаться тоже разных размеров. Полсе, получаем некоторый вектор, поверх которого добавляем полносвязные слои.

### Инициализация

<img src='img/lecture09/4.png'>

* CNN-rand - случайно инициализируем векторы для кажого слова.
* CNN-static - векторы из word2vec, без обучения.
* CNN-non-static - векторы из word2vec c дообучения.

### Недостатки

* Ищем выразительные «локальные» комбинации
* Не пытаемся понять смысл текста в целом